# Мастер и Маргарита. Разбиение на предложения

# Загрузка данных и преодобработка

In [384]:
import re

In [385]:
book = 'master_and_margo.txt'
sample = 'sample.txt'
sample_init = 'sample_init.txt'

Отрывок из текста был разбит вручную. В файле sample.txt каждое новое предложение написано с новой строки.

In [387]:
#text - полный текст
#sample_text - отрывок, разбитый на предложения
#init_text - исходный отрывок
with open(book, encoding='utf-8') as f:
    text = f.read()
with open(sample, encoding = 'utf-8') as f:
    sample_text = f.readlines()
with open(sample_init, encoding = 'utf-8') as f:
    init_text = f.read()

In [388]:
#Каждый текст был почищен от лишних символом; если подряд шло два пробела, они были заменены на один
for i in range(len(sample_text)):
    sample_text[i] = re.sub(r'\xa0', ' ', sample_text[i])
    sample_text[i] = re.sub(r'  ', ' ', sample_text[i])
    sample_text[i] = re.sub(r'\n', '', sample_text[i])

init_text = re.sub(r'\n', ' ', init_text)
text =re.sub(r'\n',' ', text)
text = re.sub(r'\t', '', text)

init_text = re.sub(r'\xa0', ' ', init_text)
text =re.sub(r'\xa0',' ', text)

text = re.sub(r'  ', ' ', text)
init_text = re.sub(r'  ', ' ', init_text)

In [393]:
print(sample_text[:5])
len(sample_text)

['На дверях комнаты N 2 было написано что-то не совсем понятное: «Однодневная творческая путевка.', 'Обращаться к М. В. Подложной».', 'Следующая дверь несла на себе краткую, но уже вовсе непонятную надпись: «Перелыгино».', 'Потом у случайного посетителя Грибоедова начинали разбегаться глаза от надписей, пестревших на ореховых теткиных дверях: «Запись в очередь на бумагу у Поклевкиной», «Касса», «Личные расчеты скетчистов»...', 'Прорезав длиннейшую очередь, начинавшуюся уже внизу в швейцарской, можно было видеть надпись на двери, в которую ежесекундно ломился народ: «Квартирный вопрос».']


39

In [391]:
init_text[:253]

'На дверях комнаты N 2 было написано что-то не совсем понятное: «Однодневная творческая путевка. Обращаться к М. В. Подложной». Следующая дверь несла на себе краткую, но уже вовсе непонятную надпись: «Перелыгино». Потом у случайного посетителя Грибоедова'

In [392]:
text[:500]

'ЧАСТЬ ПЕРВАЯ ...Так кто ж ты, наконец? – Я – часть той силы, что вечно хочет зла и вечно совершает благо.   Гете. «Фауст»  Глава 1 Никогда не разговаривайте с неизвестными Однажды весною, в час небывало жаркого заката, в Москве, на Патриарших прудах, появились два гражданина. Первый из них, одетый в летнюю серенькую пару, был маленького роста, упитан, лыс, свою приличную шляпу пирожком нес в руке, а на хорошо выбритом лице его помещались сверхъестественных размеров очки в черной роговой оправе. '

Длина открывка, с которым работаем - 39 предложений. 

# Задаем метрики

За основу были взяты метрики из ноутбука с семинара(https://github.com/mannefedov/compling_nlp_hse_course/blob/master/notebooks/Sentence_tokenizer.ipynb).
Далее идет сравнение по количеству предложений в исходном тексте и в итоговом. В функцию передается параметр funct='regexp' если анализируем работу регулярных выражений или funct='pkt' если анализируем PunktSentenceTokenizer.

In [361]:
def metrics(gold, funct, regexp=None):
    tp = 0
    fp = 0
    fn = 0

    
    if funct=='regexp':
    
        for sent in gold:
            if len(re.split(regex, sent)) == 1:
                tp += 1
            else:
                fp += 1

        for i in range(len(gold)-1):
            sent1, sent2 = gold[i], gold[i+1]
            sent = ' '.join([sent1, sent2])
            if len(re.split(regex, sent)) == 2:
                tp += 1
            else:
                fn += 1

    
    if funct =='pkt':
        for sent in gold:
            if len(tokenizer.tokenize(sent)) == 1:
                tp += 1
            else:
                fp += 1

        for i in range(len(gold)-1):
            sent1, sent2 = gold[i], gold[i+1]
            sent = ' '.join([sent1, sent2])
            if len(tokenizer.tokenize(sent)) == 2:
                tp += 1
            else:
                fn += 1

    precision = (tp/(tp+fp))
    recall = (tp/(tp+fn))
    f1 = 2*(precision*recall)/(precision+recall)
    
    return precision, recall, f1

def print_metrics(p,r,f):
    print('Precision - ', p)
    print('Recall - ', r)
    print('F1 - ', f)

# Простое разбиение части текста

In [395]:
regex =r'(?<=[.?!]) (?=[А-Я])'
simple = re.split(regex, init_text)
simple[:10]   

['На дверях комнаты N 2 было написано что-то не совсем понятное: «Однодневная творческая путевка.',
 'Обращаться к М.',
 'В.',
 'Подложной».',
 'Следующая дверь несла на себе краткую, но уже вовсе непонятную надпись: «Перелыгино».',
 'Потом у случайного посетителя Грибоедова начинали разбегаться глаза от надписей, пестревших на ореховых теткиных дверях: «Запись в очередь на бумагу у Поклевкиной», «Касса», «Личные расчеты скетчистов»...',
 'Прорезав длиннейшую очередь, начинавшуюся уже внизу в швейцарской, можно было видеть надпись на двери, в которую ежесекундно ломился народ: «Квартирный вопрос».',
 'За квартирным вопросом открывался роскошный плакат, на котором изображена была скала, а по гребню ее ехал всадник в бурке и с винтовкой за плечами.',
 'Пониже – пальмы и балкон, на балконе – сидящий молодой человек с хохолком, глядящий куда-то ввысь очень-очень бойкими глазами и держащий в руке самопишущее перо.',
 'Подпись: «Полнообъемные творческие отпуска от двух недель (рассказ-новелл

In [396]:
len_simple = len(simple)
len_simple

33

In [397]:
p_simple, r_simple, f_simple = metrics(sample_text, regexp=regex, funct='regexp')
print_metrics(p_simple, r_simple, f_simple)

Precision -  0.9850746268656716
Recall -  0.868421052631579
F1 -  0.923076923076923


Мы видим, что простого разбиения не абсолютно не достаточно. Оно не справляется с инициалами ('Обращаться к М.','В.','Подложной».' - 3 предложения вместо одного). Также, такое разбиение не учитывает диалоги, прямую речь. В результате вместо 39 предложений мы получаем 33 (и это учитывая, что, как минимум, 2 лишних предложения были добавлены из-за инициалов).

# Обучаем PunktSentenceTokenizer на всем тексте

In [398]:
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

In [399]:
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train(text)

In [367]:
tokenizer = PunktSentenceTokenizer(trainer.get_params())
print(tokenizer._params.abbrev_types)

{'ус', '«м»', 'э.»', '«н', 'м'}


# Применяем PunktSentenceTokenizer для части текста

In [368]:
res = tokenizer.tokenize(init_text)

In [400]:
p_pst, r_pst, f_pst = metrics(sample_text, funct='pkt')
print_metrics(p_pst, r_pst, f_pst)

Precision -  0.9722222222222222
Recall -  0.9333333333333333
F1 -  0.9523809523809524


In [401]:
len_pst = len(res)
len_pst

40

Применения PunktSentenceTokenizer дало намного лучший результат. Однако, не идеальный. Вместо 39 предложений мы получили 40.

In [403]:
#Сравним полученные метрики:
import pandas as pd
data = [[p_simple, r_simple, f_simple, len_simple],
        [p_pst, r_pst, f_pst, len_pst]]
pd.DataFrame(data,  columns = ['Precision', 'Recall', 'F1', 'Length(True=39)'], index = ['Simple RegExpr', 'PunktSentenceTokenizer'])

,Precision,Recall,F1,Length(True=39)
Simple RegExpr,0.985075,0.868421,0.923077,33
PunktSentenceTokenizer,0.972222,0.933333,0.952381,40


Заметно улучшение качества разбиения. Хоть точность и снизилась на 0.1, полнота и F-мера показывают значительное улучшения качества работы сегментатора.

# Работа с ошибками

Рассмотрим подробнее ошибочные случаи (текст был вручную сравнен с идеальным разбиением):

In [404]:
res[3]#Здесь нет разделения после многоточия, хотя оно должно быть.

'Потом у случайного посетителя Грибоедова начинали разбегаться глаза от надписей, пестревших на ореховых теткиных дверях: «Запись в очередь на бумагу у Поклевкиной», «Касса», «Личные расчеты скетчистов»... Прорезав длиннейшую очередь, начинавшуюся уже внизу в швейцарской, можно было видеть надпись на двери, в которую ежесекундно ломился народ: «Квартирный вопрос».'

In [405]:
res[23:25]#Здесь прямая речь отделена от слов автора, хотя так не должно быть.

['– Умеешь ты жить, Амвросий!',
 '– со вздохом отвечал тощий, запущенный, с карбункулом на шее Фока румяногубому гиганту, золотистоволосому, пышнощекому Амвросию-поэту.']

In [406]:
res[35:37]#Аналогично

['Оревуар, Фока!', '– и, напевая, Амвросий устремлялся к веранде под тентом.']

Таким образом, добавив какие-либо сокращения, эти ошибки не исчезнут.

In [407]:
#исправляем случаи с многоточием
res2 = []
regellipsis = r'(?<=\.{3}) (?=[А-Я])'
for r in res:
    if '...' in r:
        splited = re.split(regellipsis, r)
        res2.extend(splited)
    else:
        res2.append(r)

In [410]:
len(res2)

41

In [411]:
res2[3:5]

['Потом у случайного посетителя Грибоедова начинали разбегаться глаза от надписей, пестревших на ореховых теткиных дверях: «Запись в очередь на бумагу у Поклевкиной», «Касса», «Личные расчеты скетчистов»...',
 'Прорезав длиннейшую очередь, начинавшуюся уже внизу в швейцарской, можно было видеть надпись на двери, в которую ежесекундно ломился народ: «Квартирный вопрос».']

In [412]:
#исправляем диалоги
res3 = []
regexdialogs='\– [а-я]'
for i in range(len(res2)-1):
    sent1, sent2 = res2[i], res2[i+1]
    if re.match(regexdialogs, sent2):
        sent = ' '.join([sent1, sent2])
        res3.append(sent)
    else:
        try:
            if res3[-1].endswith(sent1):
                    pass
            else:
                res3.append(sent1)
        except:
            res3.append(sent1)
if res2[-1].startswith('- '):
    pass
else:
    res3.append(res2[-1])

In [413]:
len(res3)

39

In [414]:
#Посмотрим на пересечение множеств идеальных предложений и полученных итоговых res3
len(set(sample_text) & set(res3)) / len(set(sample_text) | set(res3))

1.0

Итого, получили идеальное разбиение путем изначальной сегментации с помощью PunktSentenceTokenizer и дальнейшими корректировками с помощью регулярных выражений.